# question 2

In [1]:
from os import path
import os
import pandas as pd
import numpy as np
import gurobipy as gb
from gurobipy import GRB
from gurobipy import quicksum
import math

In [2]:
work_dir = os.getcwd()
data_path = path.join(work_dir, 'data')
data_file = path.join(data_path, 's1.xlsx')

In [3]:
Demand_df = pd.read_excel(data_file, sheet_name='Demand', index_col='Product')
Init_df = pd.read_excel(data_file, sheet_name='Initial inventory', index_col='Product')
ShippingCost_df = pd.read_excel(data_file, sheet_name='Shipping cost', index_col='Product')
InTransit_df = pd.read_excel(data_file, sheet_name='In-transit', index_col='Product')
CBM_df = pd.read_excel(data_file, sheet_name='Size', index_col='Product')
PriceAndCost_df = pd.read_excel(data_file, sheet_name='Price and cost', index_col='Product')
Shortage_df = pd.read_excel(data_file, sheet_name='Shortage', index_col='Product')
VendorProduct_df = pd.read_excel(data_file, sheet_name='Vendor-Product', index_col='Product') - 1
VendorCost_df = pd.read_excel(data_file, sheet_name='Vendor cost', index_col='Vendor')
Bound_df = pd.read_excel(data_file, sheet_name='Bounds', index_col='Product')
Conflict_df = pd.read_excel(data_file, sheet_name='Conflict', index_col='Conflict') - 1

N, M = Demand_df.shape
K = 3 #(express delivery, air freight, Ocean freight)
V = VendorCost_df.shape[0]
# ContainerCap = 30 #Case 1 data
# ContainerCost = 2750 #Case 1 data
ContainerCap = 0.5 #Case 1 data
ContainerCost = 1500 #Case 1 data

Demand_ij = Demand_df.to_numpy()
Init_i = Init_df.to_numpy().squeeze()
BuyCost_i = PriceAndCost_df['Purchasing cost'].to_numpy()
HoldCost_i = PriceAndCost_df['Holding cost'].to_numpy()
Transit_ij = InTransit_df.to_numpy()
ShipFixedCost_k = np.array([100, 80, 50]) #Case 1 data
ShipVarCost_ik = np.concatenate(( ShippingCost_df.to_numpy(), np.zeros((N,1)) ), axis=1)
CBM_i = CBM_df.to_numpy().squeeze()
LostSaleCost_i = Shortage_df['Lost sales'].to_numpy()
BackOrderCost_i = Shortage_df['Backorder'].to_numpy()
BackOrderProb_i = Shortage_df['Backorder percentage'].to_numpy()
VendorFixedCost_v = VendorCost_df.to_numpy().squeeze()
MinOrder_i = Bound_df.to_numpy().squeeze()
ConflictPair_alpha = Conflict_df.to_numpy()
ProductVendor_i = VendorProduct_df.to_numpy().squeeze()
M_big = np.sum(Demand_ij) + sum(Init_i)

In [4]:
#model
instance_name = 's1'
model = gb.Model(name=instance_name)

Academic license - for non-commercial use only - expires 2021-05-20
Using license file /Users/eason/gurobi.lic


In [5]:
#decision variable
x = model.addVars(N, M, K, vtype=GRB.INTEGER,name='x')
Abin = model.addVars(M, K, vtype=GRB.BINARY, name='Abin')
ContainerCnt = model.addVars(M, vtype=GRB.INTEGER, name='ContainerCnt')
StockLevel = model.addVars(N, M, vtype=GRB.CONTINUOUS, name='StockLevel')
Shortage = model.addVars(N, M, vtype=GRB.CONTINUOUS, name='Shortage')
Bbin = model.addVars(N, M, vtype=GRB.BINARY, name='Bbin')
Cbin = model.addVars(N, M, vtype=GRB.BINARY, name='Cbin')
Dbin = model.addVars(V, M, vtype=GRB.BINARY, name='Dbin')

In [6]:
#Expr
VolumeInOceanExpr_j = [
gb.LinExpr(
    quicksum(x[i,j,2] * CBM_i[i] for i in range(N))
)    
for j in range(M)]

BackOrderCntExpr_ij = [
    [
        gb.LinExpr( Shortage[i,j] * BackOrderProb_i[i] )
    for j in range(M)]
for i in range(N)]

LostSaleCntExpr_ij = [
    [
        gb.LinExpr( Shortage[i,j] * (1 - BackOrderProb_i[i]) )
    for j in range(M)]
for i in range(N)]

In [7]:
#obj function
TotalPurchaseCost = gb.LinExpr(
    quicksum(
        quicksum(
            quicksum(
                x[i,j,k] * BuyCost_i[i]
            for i in range(N))
        for j in range(M))
    for k in range(K))
)

TotalShipFixedCost = gb.LinExpr(
    quicksum(
        quicksum(
            Abin[j,k] * ShipFixedCost_k[k]
        for k in range(K))
    for j in range(M))
)

TotalShipVarCost = gb.LinExpr(
    quicksum(
        quicksum(
            quicksum(
                x[i,j,k] * ShipVarCost_ik[i,k]
            for k in range(K))
        for j in range(M))
    for i in range(N))
)

TotalHoldingCost = gb.LinExpr(
    quicksum(
        quicksum(
            StockLevel[i,j] * HoldCost_i[i]
        for i in range(N))
    for j in range(M))
)

TotalContainerCost = gb.LinExpr(
    quicksum(ContainerCnt[j] for j in range(M)) * ContainerCost
)

TotalBackOrderCost = gb.LinExpr(
    quicksum(
        quicksum(
            BackOrderCntExpr_ij[i][j] * BackOrderCost_i[i]
        for i in range(N))
    for j in range(M))
)

TotalLostSaleCost = gb.LinExpr(
    quicksum(
        quicksum(
            LostSaleCntExpr_ij[i][j] * LostSaleCost_i[i]
        for i in range(N))
    for j in range(M))
)

TotalVendorFixedCost = gb.LinExpr(
    quicksum(
        quicksum(
            Dbin[v,j] * VendorFixedCost_v[v]
        for v in range(V))
    for j in range(M))
)

In [8]:
#set obj function
model.setObjective(
    TotalPurchaseCost +
    TotalShipFixedCost + 
    TotalShipVarCost + 
    TotalHoldingCost + 
    TotalContainerCost + 
    TotalBackOrderCost + 
    TotalLostSaleCost +
    TotalVendorFixedCost
)

In [9]:
#Contrain

#let Abin to behave correctly
_ = model.addConstrs(
    quicksum(x[i,j,k] for i in range(N)) / M_big <= Abin[j,k]
for j in range(M)
for k in range(K))

#let ContainerCnt to behave correctly
_ = model.addConstrs(
    VolumeInOceanExpr_j[j] / ContainerCap <= ContainerCnt[j]
for j in range(M))

#let Stocklevle & Shortage behave correctly
_ = model.addConstrs(StockLevel[i,0] - Shortage[i,0] == Init_i[i] - Demand_ij[i][0] for i in range(N))
_ = model.addConstrs(StockLevel[i,1] - Shortage[i,1] == StockLevel[i,0] + x[i,0,0] + Transit_ij[i][1] - Demand_ij[i][1] - Shortage[i,0] * BackOrderProb_i[i] for i in range(N))
_ = model.addConstrs(StockLevel[i,2] - Shortage[i,2] == StockLevel[i,1] + x[i,1,0] + x[i,0,1] + Transit_ij[i][2] - Demand_ij[i][2] - Shortage[i,1] * BackOrderProb_i[i] for i in range(N))
_ = model.addConstrs(StockLevel[i,j] - Shortage[i,j] == StockLevel[i,j-1] + x[i,j-1,0] + x[i,j-2,1] + x[i,j-3,2] - Demand_ij[i][j] - Shortage[i,j-1] * BackOrderProb_i[i] for i in range(N) for j in range(3,M))

_ = model.addConstrs(StockLevel[i,j] <= M_big * (1-Bbin[i,j]) for i in range(N) for j in range(M))
_ = model.addConstrs(Shortage[i,j] <= M_big * Bbin[i,j] for i in range(N) for j in range(M))

#let Cbin behave correctly
_ = model.addConstrs(
    quicksum(x[i,j,k] for k in range(K)) / M_big <= Cbin[i,j]
for i in range(N)
for j in range(M))

#let Dbin behave correctly
_ = model.addConstrs(
    quicksum(
        quicksum(x[i,j,k] for k in range(K))
    for i in range(N) if ProductVendor_i[i] == v) / M_big <= Dbin[v,j]
for v in range(V)
for j in range(M))

#Minumin order Bound
_ = model.addConstrs(
    Cbin[i,j] * MinOrder_i[i] <= quicksum(x[i,j,k] for k in range(K))
for i in range(N)
for j in range(M))

#conflict
_ = model.addConstrs(
    Cbin[a,j] + Cbin[b,j] <= 1
for j in range(M)
for a, b in ConflictPair_alpha)

In [10]:
model.optimize()

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 13390 rows, 18356 columns and 65817 nonzeros
Model fingerprint: 0xb0371461
Variable types: 5200 continuous, 13156 integer (5330 binary)
Coefficient statistics:
  Matrix range     [3e-06, 4e+05]
  Objective range  [1e+01, 2e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+05]
Presolve removed 4117 rows and 3057 columns
Presolve time: 0.22s
Presolved: 9273 rows, 15299 columns, 51451 nonzeros
Variable types: 4274 continuous, 11025 integer (3407 binary)
Found heuristic solution: objective 3.712039e+09

Root relaxation: objective 1.292218e+09, 6711 iterations, 0.07 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1.2922e+09    0  377 3.7120e+09 1.2922e+09  65.2%     -    0s
H    0     0             

In [78]:
order = np.zeros((N+1,K+1,M+1), dtype=np.float32)
for i in range(N):
    for j in range(K):
        for t in range(M):
            order[i+1,j+1,t+1] = (x[i,t,j].x)

In [83]:
from check_valid import check_feasibility_and_calculate_objective_value

In [88]:
check_feasibility_and_calculate_objective_value('./data/s1.xlsx', order)

TotalPurchaseCost    :1230662600.00
TotalShipFixedCost   :5420.00
TotalShipVarCost     :13019025.00
TotalHoldingCost     :12664881.12
TotalContainerCost   :55000.00
TotalBackOrderCost   :16816197.12
TotalLostSaleCost    :19315474.00
TotalVendorFixedCost :37500.00
objective-value      :1292576097.23


(True, 1292576097.235)

In [87]:
print(f'TotalPurchaseCost    :{TotalPurchaseCost.getValue():.2f}')
print(f'TotalShipFixedCost   :{TotalShipFixedCost.getValue():.2f}')
print(f'TotalShipVarCost     :{TotalShipVarCost.getValue():.2f}')
print(f'TotalHoldingCost     :{TotalHoldingCost.getValue():.2f}')
print(f'TotalContainerCost   :{TotalContainerCost.getValue():.2f}')
print(f'TotalBackOrderCost   :{TotalBackOrderCost.getValue():.2f}')
print(f'TotalLostSaleCost    :{TotalLostSaleCost.getValue():.2f}')
print(f'TotalVendorFixedCost :{TotalVendorFixedCost.getValue():.2f}')
print(f'objective-value      :{model.ObjVal:.2f}')

TotalPurchaseCost    :1230662600.00
TotalShipFixedCost   :5420.00
TotalShipVarCost     :13019025.00
TotalHoldingCost     :12664881.12
TotalContainerCost   :202500.00
TotalBackOrderCost   :16816197.12
TotalLostSaleCost    :19315474.00
TotalVendorFixedCost :37500.00
objective-value      :1292723597.23


In [25]:
os.makedirs('solution', exist_ok=True)

solution_file_path = path.join(work_dir, 'solution', 'q1.xlsx')

In [26]:
with pd.ExcelWriter(solution_file_path, engine='xlsxwriter') as writer:
    print('Express delivery')
    df1 = pd.DataFrame([[int(x[i,j,0].x) for j in range(M)] for i in range(N)], columns=range(M))
    display(df1)
    df1.to_excel(writer, sheet_name='Express delivery')
    print('Air frieght')
    df2 = pd.DataFrame([[int(x[i,j,1].x) for j in range(M)] for i in range(N)], columns=range(M))
    display(df2)
    df2.to_excel(writer, sheet_name='Air frieght')
    print('Ocean frieght')
    df3 = pd.DataFrame([[int(x[i,j,2].x) for j in range(M)] for i in range(N)], columns=range(M))
    display(df3)
    df3.to_excel(writer, sheet_name='Ocean frieght')
    print('StockLevel')
    df = pd.DataFrame([[StockLevel[i,j].x for j in range(M)] for i in range(N)], columns=range(M))
    display(df)
    df.to_excel(writer, sheet_name='StockLevel')
    print('Shortage')
    df = pd.DataFrame([[Shortage[i,j].x for j in range(M)] for i in range(N)], columns=range(M))
    display(df)
    df.to_excel(writer, sheet_name='Shortage')

Express delivery


,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,7,0,0,0,4,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0,0,0,8,0,44,0,70,163,0,...,0,0,0,70,108,102,0,0,0,0
96,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
97,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Air frieght


,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,0,0,0,38,79,12,151,165,142,163,...,49,0,67,148,118,166,66,0,0,0
1,0,0,0,0,0,0,23,0,13,0,...,0,4,0,0,0,0,0,0,0,0
2,0,0,82,200,66,199,101,0,270,220,...,0,196,61,0,174,332,0,187,0,0
3,0,0,97,155,153,75,108,139,97,182,...,0,0,0,0,0,0,0,0,0,0
4,0,0,60,53,67,71,158,56,203,0,...,0,0,193,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0,0,0,113,70,156,143,0,125,138,...,197,251,132,0,0,136,255,283,0,0
96,0,0,0,0,0,190,55,0,100,264,...,136,144,52,101,145,317,66,217,0,0
97,0,0,0,0,0,0,0,0,0,0,...,0,292,30,0,147,178,0,0,0,0
98,0,0,52,143,62,129,67,137,10,192,...,28,38,63,115,151,74,0,10,0,0


Ocean frieght


,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,60,241,145,0,185,0,157,197,...,152,57,242,0,227,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
96,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
97,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


StockLevel


,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,662.0,607.0,4.350000e+02,2.410000e+02,1.470000e+02,0.0,0.0,0.0,-0.0,0.0,...,0.000000e+00,0.0,0.0,0.0,0.0,-0.0,-0.0,0.0,-0.0,0.0
1,410.0,357.0,2.890000e+02,1.040000e+02,9.100000e+01,15.0,0.0,0.0,0.0,0.0,...,4.547474e-14,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,346.0,167.0,1.660000e+02,1.170000e+02,0.000000e+00,0.0,0.0,0.0,0.0,0.0,...,0.000000e+00,0.0,0.0,-0.0,0.0,0.0,0.0,0.0,0.0,-0.0
3,208.0,258.0,1.800000e+02,4.900000e+01,0.000000e+00,0.0,-0.0,0.0,0.0,-0.0,...,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,365.0,303.0,2.200000e+02,1.300000e+02,0.000000e+00,4.0,0.0,0.0,0.0,0.0,...,1.000000e+00,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,481.0,322.0,2.560000e+02,8.400000e+01,0.000000e+00,0.0,0.0,0.0,0.0,0.0,...,1.421085e-14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96,761.0,582.0,5.600000e+02,5.080000e+02,3.960000e+02,101.0,9.0,0.0,0.0,0.0,...,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
97,709.0,638.0,7.370000e+02,6.830000e+02,5.700000e+02,461.0,449.0,194.0,194.0,170.0,...,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98,213.0,253.0,1.840000e+02,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,...,-0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0


Shortage


,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,154.0,269.8
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,249.0,290.0,290.0,477.0,706.0,803.0,1029.0,1089.0,1089.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,13.0,350.0,363.0,363.0,398.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
97,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,258.0
98,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [81]:
print(f'TotalPurchaseCost    :{TotalPurchaseCost.getValue():.2f}')
print(f'TotalShipFixedCost   :{TotalShipFixedCost.getValue():.2f}')
print(f'TotalShipVarCost     :{TotalShipVarCost.getValue():.2f}')
print(f'TotalHoldingCost     :{TotalHoldingCost.getValue():.2f}')
print(f'TotalContainerCost   :{TotalContainerCost.getValue():.2f}')
print(f'TotalBackOrderCost   :{TotalBackOrderCost.getValue():.2f}')
print(f'TotalLostSaleCost    :{TotalLostSaleCost.getValue():.2f}')
print(f'TotalVendorFixedCost :{TotalVendorFixedCost.getValue():.2f}')
print(f'objective-value      :{model.ObjVal:.2f}')

TotalPurchaseCost    :1230662600.00
TotalShipFixedCost   :5420.00
TotalShipVarCost     :13019025.00
TotalHoldingCost     :12664881.12
TotalContainerCost   :202500.00
TotalBackOrderCost   :16816197.12
TotalLostSaleCost    :19315474.00
TotalVendorFixedCost :37500.00
objective-value      :1292723597.23


In [28]:
result_df = pd.DataFrame(columns=[
    'TotalPurchaseCost',
    'TotalShipFixedCost',
    'TotalShipVarCost',
    'TotalHoldingCost',
    'TotalContainerCost',
    'TotalBackOrderCost',
    'TotalLostSaleCost',
    'TotalVendorFixedCost',
    'objective-value',
])

# result_df.append({
#     'TotalPurchaseCost': str(round(TotalPurchaseCost.getValue(), 2)),
#     'TotalShipFixedCost': str(round(TotalShipFixedCost.getValue(), 2)),
#     'TotalShipVarCost': str(round(TotalShipVarCost.getValue(), 2)),
#     'TotalHoldingCost': str(round(TotalHoldingCost.getValue(), 2)),
#     'TotalContainerCost': str(round(TotalContainerCost.getValue(), 2)),
#     'TotalBackOrderCost': str(round(TotalBackOrderCost.getValue(), 2)),
#     'TotalLostSaleCost': str(round(TotalLostSaleCost.getValue(), 2)),
#     'TotalVendorFixedCost': str(round(TotalVendorFixedCost.getValue(), 2)),
#     'objective-value': model.ObjVal,
# }, ignore_index=True)

result_df.loc[0] = [
    str(round(TotalPurchaseCost.getValue(), 2)),
    str(round(TotalShipFixedCost.getValue(), 2)),
    str(round(TotalShipVarCost.getValue(), 2)),
    str(round(TotalHoldingCost.getValue(), 2)),
    str(round(TotalContainerCost.getValue(), 2)),
    str(round(TotalBackOrderCost.getValue(), 2)),
    str(round(TotalLostSaleCost.getValue(), 2)),
    str(round(TotalVendorFixedCost.getValue(), 2)),
    str(round(model.ObjVal, 2)),
]
result_df


,TotalPurchaseCost,TotalShipFixedCost,TotalShipVarCost,TotalHoldingCost,TotalContainerCost,TotalBackOrderCost,TotalLostSaleCost,TotalVendorFixedCost,objective-value
0,1230662600.0,5420.0,13019025.0,12664881.12,202500.0,16816197.12,19315474.0,37500.0,1292723597.23
